In [1]:
si_id = [1,6,4,2,5,3,7,8,31,32,33,34,35,36,37,38,39]

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설3'
# 파일 저장
f_dir = 'c:\\공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
cnttt = 50
page_cnt = math.ceil(cnttt/10)

for j in range(0,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)

    r_cnt= soup.find('span', {"id" = "totalCnt"}).get_text( )
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
            
            if no > cnttt :
                break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]

            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            cnt += 1
        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
        
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 1839 건
실제 최종 출력 건수 50
no 1 :  아르코미술관
서울 종로구
서울특별시 종로구 동숭길 3


no 2 :  서울특별시교육청 정독도서관
서울 종로구
서울특별시 종로구 북촌로5길 48


no 3 :  춘원당한의약박물관
서울 종로구
서울특별시 종로구 돈화문로9길 27


no 4 :  동림매듭박물관
서울 종로구
서울특별시 종로구 북촌로12길 10


no 5 :  북촌문화센터
서울 종로구
서울특별시 종로구 계동길 37


no 6 :  환기미술관
서울 종로구
서울특별시 종로구 자하문로40길 63


no 7 :  목인박물관 목석원
서울 종로구
서울특별시 종로구 창의문로5길 46-1


no 8 :  도산안창호기념관
서울 강남구
서울특별시 강남구 도산대로45길 20


no 9 :  용산공예관
서울 용산구
서울특별시 용산구 이태원로 274


no 10 :  서대문자연사박물관
서울 서대문구
서울특별시 서대문구 연희로32길 51


no 11 :  매헌 윤봉길의사 기념관
서울 서초구
서울특별시 서초구 매헌로 99


no 12 :  양천문화원
서울 양천구
서울특별시 양천구 목동서로 367


no 13 :  강동청소년회관
서울 강동구
서울특별시 강동구 양재대로138길 41


no 14 :  식민지역사박물관
서울 용산구
서울특별시 용산구 청파로47다길 27


no 15 :  코엑스 아쿠아리움
서울 강남구
서울특별시 강남구 영동대로 513


no 16 :  서울특별시교육청 개포도서관
서울 강남구
서울특별시 강남구 선릉로4길 30


no 17 :  지식재산전문도서관
서울 강남구
서울특별시 강남구 테헤란로 131


no 18 :  서울특별시교육청 강남도서관
서울 강남구
서울특별시 강남구 선릉로116길 45


no 19 :  암사동 선사주거지 원시생활전시관
서울 강동구
서울특별시 강동구 올림픽로 875


no 20 :  서울특별시교육청 고덕평생학습관
서울 강동구
서울특별시 강동구 고덕로 295


no 21 :  별마당도

AttributeError: 'NoneType' object has no attribute 'find'

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설3 부산부터'
# 파일 저장
f_dir = 'c:\\tmp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
cnttt = 50
page_cnt = math.ceil(cnttt/10)

for j in range(1,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)

    r_cnt= soup.find('span', {"id":"totalCnt"}).get_text( )
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
            
            if no > cnttt :
                break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]

            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            cnt += 1
        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
        
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 1839 건
실제 최종 출력 건수 50
no 1 :  부산 트릭아이뮤지엄
부산 중구
부산광역시 중구 대청로126번길 12


no 2 :  복천박물관(부산)
부산 동래구
부산광역시 동래구 복천로 63


no 3 :  부산문화회관
부산 남구
부산광역시 남구 유엔평화로76번길 1


no 4 :  유엔평화기념관
부산 남구
부산광역시 남구 홍곡로320번길 106


no 5 :  부산시립미술관
부산 해운대구
부산광역시 해운대구 APEC로 58


no 6 :  국립부산국악원
부산 부산진구
부산광역시 부산진구 국악로 2


no 7 :  북구문화빙상센터 공연장
부산 북구
부산광역시 북구 금곡대로46번길 50


no 8 :  부산시민회관
부산 동구
부산광역시 동구 자성로133번길 16


no 9 :  요산문학관
부산 금정구
부산광역시 금정구 팔송로 60-6


no 10 :  부산 해운대도서관
부산 해운대구
부산광역시 해운대구 양운로 183


no 11 :  동아대학교 석당박물관(부산)
부산 서구
부산광역시 서구 구덕로 225


no 12 :  부산현대미술관
부산 사하구
부산광역시 사하구 낙동남로 1191


no 13 :  부산광역시 영도도서관
부산 영도구
부산광역시 영도구 함지로79번길 6


no 14 :  정관박물관
부산 기장군
부산광역시 기장군 정관읍 정관중앙로 122


no 15 :  임시수도기념관
부산 서구
부산광역시 서구 임시수도기념로 45


no 16 :  국립해양박물관
부산 영도구
부산광역시 영도구 해양로301번길 45


no 17 :  부산광역시립 명장도서관
부산 동래구
부산광역시 동래구 명안로46번길 35


no 18 :  부산광역시립 중앙도서관
부산 중구
부산광역시 중구 망양로193번길 146


no 19 :  부산박물관
부산 남구
부산광역시 남구 유엔평화로 63


no 20 :  부산광역시립 연산도서관
부산 연제구
부산광역시 연제구 고분로191번길 16


no 21 :  동삼동패총전시관
부산 영도구

AttributeError: 'NoneType' object has no attribute 'get_text'

In [3]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설3 대구부터'
# 파일 저장
f_dir = 'c:\\tmp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
cnttt = 50
page_cnt = math.ceil(cnttt/10)
f = open(ff_name, 'a', encoding='UTF-8')

for j in range(2,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            
            if no > cnttt :
                break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]

            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            cnt += 1
        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
        
# tour = pd.DataFrame()
# tour['no']=no2
# tour['title']=contents
# tour['area_cd']=area_CD2
# tour['addr']=addr2

# # csv 형태로 저장하기
# tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# # 엑셀 형태로 저장하기
# tour.to_excel(fx_name,index=False)

# e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
# t_time = e_time - s_time

# # Step 9. 요약정보 보여주기
# print("\n") 
# print("=" *80)
# print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
# print("파일 저장 완료: txt 파일명 : %s " %ff_name)
# print("파일 저장 완료: csv 파일명 : %s " %fc_name)
# print("파일 저장 완료: xls 파일명 : %s " %fx_name)
# print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 61 건
실제 최종 출력 건수 50
no 1 :  대구예술발전소
대구 중구
대구광역시 중구 달성로22길 31-12


no 2 :  향촌문화관
대구 중구
대구광역시 중구 중앙대로 449


no 3 :  대구광역시립중앙도서관
대구 중구
대구광역시 중구 공평로10길 25


no 4 :  수성아트피아
대구 수성구
대구광역시 수성구 무학로 180


no 5 :  패션주얼리전문타운
대구 중구
대구광역시 중구 경상감영길 176


no 6 :  에코한방웰빙체험관
대구 중구
대구광역시 중구 남성로 24


no 7 :  근대문화체험관 계산예가
대구 중구
대구광역시 중구 서성로 6-1


no 8 :  팔공문화원
대구 동구
대구광역시 동구 팔공로197길 25


no 9 :  아양아트센터(구, 동구문화체육회관)
대구 동구
대구광역시 동구 효동로2길 24


no 10 :  달성 한일우호관
대구 달성군
대구광역시 달성군 가창면 우록길 206


no 11 :  행복북구문화재단 어울아트센터
대구 북구
대구광역시 북구 구암로 47


no 12 :  대구오페라하우스
대구 북구
대구광역시 북구 호암로 15


no 13 :  대구미술관
대구 수성구
대구광역시 수성구 미술관로 40


no 14 :  국립대구박물관
대구 수성구
대구광역시 수성구 청호로 321


no 15 :  서구어린이도서관
대구 서구
대구광역시 서구 문화로 123


no 16 :  대구서구문화회관
대구 서구
대구광역시 서구 당산로 403


no 17 :  대구 아쿠아리움
대구 동구
대구광역시 동구 동부로 149


no 18 :  수창청춘맨숀
대구 중구
대구광역시 중구 달성로22길 27


no 19 :  대구광역시어린이회관
대구 수성구
대구광역시 수성구 동대구로 176


no 20 :  향토역사관
대구 중구
대구광역시 중구 달성공원로 35


no 21 :  대구문화예술회관
대구 달서구
대구광역시 달서구 공원순환로 201


no 22 :  수성구립 고산도서관
대구 수성구
대

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"4"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [10]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설3 인천부터 진짜'
# 파일 저장
f_dir = 'c:\\공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(3,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 50
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
            if no >= cnttt :
                break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))

        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
# tour = pd.DataFrame()
# tour['no']=no2
# tour['title']=contents
# tour['area_cd']=area_CD2
# tour['addr']=addr2

# # csv 형태로 저장하기
# tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# # 엑셀 형태로 저장하기
# tour.to_excel(fx_name,index=False)

# e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
# t_time = e_time - s_time

# # Step 9. 요약정보 보여주기
# print("\n") 
# print("=" *80)
# print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
# print("파일 저장 완료: txt 파일명 : %s " %ff_name)
# print("파일 저장 완료: csv 파일명 : %s " %fc_name)
# print("파일 저장 완료: xls 파일명 : %s " %fx_name)
# print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 74 건
실제 최종 출력 건수 50
no 1 :  국립생물자원관
인천 서구
인천광역시 서구 환경로 42


no 2 :  인천 검단선사박물관
인천 서구
인천광역시 서구 고산후로121번길 7


no 3 :  청라호수도서관
인천 서구
인천광역시 서구 크리스탈로 59


no 4 :  인천동구청소년수련관
인천 동구
인천광역시 동구 솔빛로 82


no 5 :  인천광역시 여성문화회관
인천 부평구
인천광역시 부평구 길주로 539


no 6 :  송암미술관
인천 미추홀구
인천광역시 미추홀구 비류대로55번길 68


no 7 :  인천문화예술회관
인천 남동구
인천광역시 남동구 예술로 149


no 8 :  인천수봉문화회관
인천 미추홀구
인천광역시 미추홀구 수봉안길 78


no 9 :  녹청자박물관
인천 서구
인천광역시 서구 도요지로 54


no 10 :  인천상륙작전기념관
인천 연수구
인천광역시 연수구 청량로 138


no 11 :  한중문화관
인천 중구
인천광역시 중구 제물량로 238


no 12 :  인천도시역사관
인천 연수구
인천광역시 연수구 인천타워대로 238


no 13 :  연수문화원
인천 연수구
인천광역시 연수구 비류대로 299


no 14 :  인천광역시 미추홀도서관
인천 남동구
인천광역시 남동구 인주대로776번길 53


no 15 :  송도해돋이도서관
인천 연수구
인천광역시 연수구 해돋이로 7


no 16 :  중구생활사전시관
인천 중구
인천광역시 중구 신포로23번길 101


no 17 :  영종역사관
인천 중구
인천광역시 중구 구읍로 63


no 18 :  짜장면박물관
인천 중구
인천광역시 중구 차이나타운로 56-14


no 19 :  소래역사관
인천 남동구
인천광역시 남동구 아암대로 1605


no 20 :  인천아트플랫폼
인천 중구
인천광역시 중구 제물량로218번길 3


no 21 :  부평 기적의도서관
인천 부평구
인천광역시 부평구 길주남로 166


no 22 :  인천 서구도서관
인천 서구


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"4"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [2]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설3 대전부터 진짜'
# 파일 저장
f_dir = 'c:\\공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(5,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 50
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
# tour = pd.DataFrame()
# tour['no']=no2
# tour['title']=contents
# tour['area_cd']=area_CD2
# tour['addr']=addr2

# # csv 형태로 저장하기
# tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# # 엑셀 형태로 저장하기
# tour.to_excel(fx_name,index=False)

# e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
# t_time = e_time - s_time

# # Step 9. 요약정보 보여주기
# print("\n") 
# print("=" *80)
# print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
# print("파일 저장 완료: txt 파일명 : %s " %ff_name)
# print("파일 저장 완료: csv 파일명 : %s " %fc_name)
# print("파일 저장 완료: xls 파일명 : %s " %fx_name)
# print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 40 건
실제 최종 출력 건수 40
no 1 :  유성문화원
대전 유성구
대전광역시 유성구 문화원로 46


no 2 :  이응노 미술관
대전 서구
대전광역시 서구 둔산대로 157


no 3 :  전통나래관
대전 동구
대전광역시 동구 철갑1길 15


no 4 :  대전 가수원도서관
대전 서구
대전광역시 서구 가수원로 91-11


no 5 :  평송청소년문화센터
대전 서구
대전광역시 서구 둔산대로 201


no 6 :  대전 갈마도서관
대전 서구
대전광역시 서구 신갈마로127번길 138


no 7 :  대전시립박물관
대전 유성구
대전광역시 유성구 도안대로 398


no 8 :  대전한밭도서관
대전 중구
대전광역시 중구 서문로 10


no 9 :  대전시립미술관
대전 서구
대전광역시 서구 둔산대로 155


no 10 :  대전어린이회관
대전 유성구
대전광역시 유성구 월드컵대로 32


no 11 :  대전선사박물관
대전 유성구
대전광역시 유성구 노은동로 126


no 12 :  대전학생교육문화원
대전 중구
대전광역시 중구 동서대로 1360


no 13 :  대전예술의전당
대전 서구
대전광역시 서구 둔산대로 135


no 14 :  대전근현대사전시관(옛 충남도청사 본관)
대전 중구
대전광역시 중구 중앙로 101


no 15 :  천연기념물센터(대전)
대전 서구
대전광역시 서구 유등로 927


no 16 :  신탄진도서관
대전 대덕구
대전광역시 대덕구 신탄진동로 64


no 17 :  대덕문화원
대전 대덕구
대전광역시 대덕구 대전로 1348


no 18 :  대덕문예회관(구, 대덕구 평생학습센터)
대전 대덕구
대전광역시 대덕구 대전로 1348


no 19 :  한밭교육박물관
대전 동구
대전광역시 동구 우암로 96


no 20 :  지질박물관
대전 유성구
대전광역시 유성구 과학로 124


no 21 :  국립중앙과학관
대전 유성구
대전광역시 유성구 대덕대로 481


no 22 :  대전통일관
대전 유성구
대전광

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"다음"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [3]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설3 대전다음'
# 파일 저장
f_dir = 'c:\\공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(6,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 50
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        elif (no == cnttt):
            break
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
# tour = pd.DataFrame()
# tour['no']=no2
# tour['title']=contents
# tour['area_cd']=area_CD2
# tour['addr']=addr2

# # csv 형태로 저장하기
# tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# # 엑셀 형태로 저장하기
# tour.to_excel(fx_name,index=False)

# e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
# t_time = e_time - s_time

# # Step 9. 요약정보 보여주기
# print("\n") 
# print("=" *80)
# print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
# print("파일 저장 완료: txt 파일명 : %s " %ff_name)
# print("파일 저장 완료: csv 파일명 : %s " %fc_name)
# print("파일 저장 완료: xls 파일명 : %s " %fx_name)
# print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 24 건
실제 최종 출력 건수 24
no 1 :  울산박물관
울산 남구
울산광역시 남구 두왕로 277


no 2 :  가족문화센터
울산 남구
울산광역시 남구 대공원로 84


no 3 :  울주문화예술회관
울산 울주군
울산광역시 울주군 범서읍 천상중앙길 36


no 4 :  현대자동차 문화회관
울산 북구
울산광역시 북구 염포로 601


no 5 :  현대예술관
울산 동구
울산광역시 동구 명덕로 10


no 6 :  울산북구문화예술회관
울산 북구
울산광역시 북구 산업로 1000


no 7 :  한마음회관
울산 동구
울산광역시 동구 바드래1길 30


no 8 :  울산암각화박물관
울산 울주군
울산광역시 울주군 두동면 반구대안길 254


no 9 :  울산남부도서관
울산 남구
울산광역시 남구 거마로13번길 6


no 10 :  울산동부도서관
울산 동구
울산광역시 동구 봉수로 505


no 11 :  태화강생태관
울산 울주군
울산광역시 울주군 범서읍 구영로 31


no 12 :  울산 옹기박물관
울산 울주군
울산광역시 울주군 온양읍 외고산3길 36


no 13 :  울산대곡박물관
울산 울주군
울산광역시 울주군 두동면 서하천전로 257


no 14 :  약사동 제방유적전시관
울산 중구
울산광역시 중구 종가14길 22-28


no 15 :  울산울주도서관
울산 울주군
울산광역시 울주군 언양읍 언양로 109


no 16 :  장생포 고래박물관
울산 남구
울산광역시 남구 장생포고래로 244


no 17 :  울산 동구문화원
울산 동구
울산광역시 동구 화진6길 4


no 18 :  울산문화예술회관
울산 남구
울산광역시 남구 번영로 200


no 19 :  울산해양박물관
울산 울주군
울산광역시 울주군 서생면 해맞이로 1251


no 20 :  KBS 울산홀
울산 남구
울산광역시 남구 번영로 212


no 21 :  울산 남구문화원
울산 남구
울산광역시 남구 번영로 224


no 22 :  울산북구문화원
울산 북구
울산

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"4"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [7]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(0,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 60
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) +'//'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
cul = pd.DataFrame()
cul['no']=no2
cul['title']=contents
cul['area_cd']=area_CD2
cul['addr']=addr2
cul['content']=view2

# csv 형태로 저장하기
cul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
cul.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 415 건
실제 최종 출력 건수 60
no 1 :  아르코미술관
서울 종로구
서울특별시 종로구 동숭길 3
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.19 ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기아르코미술관은 1979년 5월 국립서울대학교 터에 한국문화예술진흥원 산하 전시지원시설인 미술회관으로 개관하였다. ''한국 문화와 예술 생산의 심장부''라고 할 수 있는 서울 대학로에 위치하고 있는 아르코미술관 건물은 한국의 대표적 건축가 김수근이 설계하였고, 그의 붉은 벽돌 건물은 대학로의 대표적 상징으로 자리 잡았다. 건축사적으로도 보존가치가 높음을 인정받아 2013년 서울 미래유산으로 등재됐다.국내에 미술관이 절대 부족하던 건립초기의 상황에서 미술회관은 각종 미술단체나 개인전을 지원하기 위한 대관공간과 기획전시공간으로 활용되었고, 공.사립미술관이 늘어난 80년대 후반부터는 미술관의 자체 기획 전시공간으로 점진적인 전환을 꾀하였다. 1990년대 말부터는 공공미술관으로서의 전문성이 강화되며 새로운 한국 현대미술의 패러다임을 바꾸어가는 선도적 기능을 수행하기 시작하였다. 2005년 이후 한국문화예술진흥원이 한국문화예술위원회로 전환되면서 아르코미술관은 예술 감독제를 도입하여 운영의 책임성과 전문성을 강화하였다. 아르코미술관은 한국의 컨템포러리 아트 및 이에 관한 담론 생산처로서, 장르나 매체간의 통섭적 사유를 구현하기 위한 실험적인 기획전과 이를 대중들과 소통할 수 있는 다양한 교육프로그램을 운영한다. 또한, 지역과 국제적 네트웍을 강화하며, 글로벌네트웍을 구축하기 위한 다양한 협업 프로그램을 운영한다. 이외에도 실험적인 신진작가들을 지원하기 위한 별도의 부대시설로서 대안적 공간인 인사미술공간을 운영하며, 아카이브와 세미나실, 프로젝트 카페를 통해 미술계의 다양한 담론 생산과 소통의 장을 마련하고 있다.1979년 미술회관으로 개관해 2005년 9월 아르코미술관으로 이름을 바꾼 이곳은 문화예

no 19 :  암사동 선사주거지 원시생활전시관
서울 강동구
서울특별시 강동구 올림픽로 875
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기암사동선사주거지는 약 6,000년전, 신석기시대 사람들이 살았던 집단취락지로, 1925년(을축년) 대홍수 때 한강변 모래언덕 지대가 패어지면서 수많은 빗살무늬 조각이 노출되어 세상에 알려졌다. 1979년 사적 제267호로 지정되었고, 1981년부터 1988년까지 유적지 발굴조사를 시작으로 총 면적 78,133㎡의 문화재 보호구역을 확대하였다. 현재 9채의 신석기시대 움집과 1채의 체험움집이 복원되었고, 신석기시대 움집터를 경화처리하여 관람할 수 있게 한 원시생활 전시관인 제 1전시관과 선사시대 전반을 이해할 수 있는 각종 판넬 및 모형 등을 전시한 원시생활 전시관인 제 2전시관을 건립하여 일반인에게 공개하고 있다.


no 20 :  서울특별시교육청 고덕평생학습관
서울 강동구
서울특별시 강동구 고덕로 295
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기고덕평생학습관은 1984년 고덕도서관으로 개관한 이래 지역주민의 정보욕구와 독서생활화에 기여해 왔다. 이제 세계화, 정보화 시대를 대비하여 서울시교육청 조례 제3561호에 근거하여 1999.7.16일자로 고덕평생학습관으로  거듭나게 되었다.  * 개관 - 1984년 08월  고덕도서관 개관


no 21 :  별마당도서관
서울 강남구
서울특별시 강남구 영동대로 513
※ 코로나바이러스감염증-19 공지사항※ 내용 : 일부 매장 임시 휴점(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기별마당 도서관은 코엑스 센트럴플라자 중심에 총 2,800㎡ 복층으로 구성되어 있다. 13m 높이 서가의 은은한 불빛이 공간 전체를 부드럽게 감싸는 편안한 서재를 컨셉으로 하여 다양한 테이블과 

no 41 :  소마미술관
서울 송파구
서울특별시 송파구 올림픽로 424
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기88서울올림픽의 문화적 성과를 재조명하고 자연과 공존하는 소통의 문화공간 조성을 위해 2004년 9월 개관하였다. 222점의 조각작품과 국내 최초의 드로잉센터와 아카이브를 설립하고 다양한 전시활동은 물론이고 사회봉사, 어린이 교육프로그램 등 국민의 삶을 풍요롭게 하는 사회기관이며 열린 문화공간으로의 역할에 충실하고 있다. 2004년 9월 미술관 개관


no 42 :  서울특별시교육청 영등포평생학습관
서울 영등포구
서울특별시 영등포구 버드나루로15길 10
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기지역주민의 삶의 질 향상과 자아실현을 위한 다양한 평생학습 프로그램을 운영하여 자기계발의 기회를 제공하고, 독서 문화정착과 지역사회 지식, 정보 문화센터의 역할을 하기 위해 많은 자료를 비치하고 이용자의 편의를 우선하여 서비스하고 있다. 또한, 지역 주민의 여가선용 및 보람과 행복감을 느낄 수 있도록 다양한 문화활동을 전개하고 있다.* 개관 - 1974년 6월 영등포 간이도서관 개관


no 43 :  국립국악원
서울 서초구
서울특별시 서초구 남부순환로 2364
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.16.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기* 전통음악과 무용을 보존·전승하고, 보급 및 발전시키고자 설립된 국립음악기관 * 각종 공연을 비롯한 학술 연구, 국악 교육, 해외 교류 등으로 국민과 함께하는 국악, 세계 속의 우리 국악을 추구한다. 신라시대 이후 전승되어온 궁중음악기관에 그 기원을 둔다. 8.15해방 후 궁중음악기관이 해체된  한때 구왕궁아악부(舊王宮雅樂部)로 존속하다가 1951년 이후 국립국악원으로 이어졌다.  

no 59 :  양천구 갈산도서관
서울 양천구
서울특별시 양천구 목동남로4길 48-6
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기양천구 교육의 메카로 우뚝 선 갈산도서관은 2014년 10월개관 이래 주민들의 사랑을 독차지하고 있는 지역명소중의 명소이다. 주요시설로는 1층에는 갈산 어린이집,2층 글고운터(유아자료실)에는 북스타트 증의 프로그램을 운영할 수 있는 유아전용공간이 마련되어 있다. 3층 글길터(종합자료실)에는 일반도서와 함께 갈산도서관만의 특성화 공간인 ''천문학 자료실''이 별도로 구비되어 있으며, 4층 글마루 열람실은 주민 누구나 자유롭게 공부하거나 독서할 수 있는 공간으로 꾸며졌다. 5층에는 주민들의 참여와 소통의 커뮤니티 공간인 북카페, 일명 책다방이 마련되었다. 이곳에서는 탁 트인 옥상정원을 보며 책과 차 한 잔의 여유로움은 물론, 이웃 간의 공감의 시간도 가질 수 있다. 이밖에 지하1층에 있는''한울관''은 무대 및 음향.조명시설이 완비된 다목적홀로 각종 저자와의 만남, 영화상영, 낭독공연 등의 행사를 개최해 주민들로부터 좋은 호응을 얻고 있다.


no 60 :  신월음악도서관
서울 양천구
서울특별시 양천구 오목로5길 34
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기책으로 하나 되는 양천구, 그 길을 신월디지털정보도서관이 함께 걸어가겠습니다.신월디지털정보도서관은 2006년 2월 지역 주민들에게 독서문화 및 지식정보제공을 위해 신월4동 복합청사에 설립된 양천구 구립도서관입나다.신월디지털정보도서관은 양천구 지역 주민들을 위한 독서 및 문화 나눔터를 지향합니다. 앞으로도 주민들의 다양한 관심사를 반영한 장서와 독서 및 문화 프로그램을 제공하면서 올바른 도서관 문화 형성에 기여하겠습니다.


전체 검색 결과 건수 : 86 건
실제 최종 출력 건수 60
no 1 :  부

no 18 :  부산광역시립 중앙도서관
부산 중구
부산광역시 중구 망양로193번길 146
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.21 ~ 별도해제시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기 * 정보화시대를 이끄는 선두주자, 부산시립중앙도서관 *세계는 빠른 속도로 변화하고 있으며 사회영역간 경계도 나라간 국경도 허물면서 사람들 의 인식조차 바뀌기를 요구하고 있다. 새로운 세기, 21세기의 키워드는 정보와 문화이다. 누가 더 많은 정보를 보다 빠르게 획득하여 문화적 부가가치를 창출하는가에 따라서 삶의 질도 달라질 수 있다. 부산시립중앙도서관은 모두가 필요한 정보제공과 문화생활 향상에 기여하는 것이 중요한 존재가치의 하나라는 인식아래 기존의 서비스 방식에서 탈피하는 새로운 서비스의 첫 단 계로서 홈페이지를 개설하였고 또한 원하는 정보를 적시에 전달하는 것과 문화생활의 터전이 되는 것이 궁극적인 목표이다. 1989년 8월 부산직할시립 중앙도서관의 설립절차를 마치고 1989년 12월 도서관 건물을 완공하였다. 1990년 4월에 개관하고 1995년 1월 부산광역시립 중앙도서관으로 명칭을 변경하였다. 1999년 1월 부산광역시립 수정분관을 인수하였다.


no 19 :  부산박물관
부산 남구
부산광역시 남구 유엔평화로 63
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.21 ~ 별도안내시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기 * 종합문화교육의 장, 부산박물관 *부산박물관은 1978년 7월 11일 개관이래 문화유산의 수집, 보존, 전시, 연구, 발굴, 조사, 박물관 교육 등 부산지역 전통문화 전승에 앞장서 왔다. 산하에 복천박물관, 동삼동패총전시관, 부산근대역사관, 임시수도기념관 등 전문박물관을 두고 있는 부산의 대표적 종합역사 박물관으로 소장유물은 총 22,000여 점이며, 그 중 1,300여 점의 유물을 상설 전시하고 있다. 제 2전시관 건립과 함께 2002년 부산의 역사와 문화

...

no 36 :  인천 화도진도서관
인천 동구
인천광역시 동구 화도진로 122
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19 ~ 별도안내시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기인천화도진도서관은 1988년 개관하여 시민문화 생활의 공간으로서 새로운 문화 창달과 지역사회 발전을 위하여 항상 노력하고 있다. 특히 우리 유서 깊은 화도진 공원 및 인 천항과 근접하여 인천개항과 관련된 모든 자료를 수집 비치하여, 향토역사 체험학습장 으로 운영되는 향토ㆍ개항문화자료관과 시각장애인을 위한 녹음,점자 자료를  자체 제 작하여 운영되고 있는 시각장애인실을 특색사업으로 운영하고 있다.* 개관일 - 1988년 10월 20일


no 37 :  가천박물관
인천 연수구
인천광역시 연수구 청량로102번길 40-9
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19 ~ 별도공지시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기가천박물관은 인천 문화에 보탬이 되고자 지난 1995년 10월 문을 열었다. 국내 사립박물관 가운데 77번째로 등록된 본 박물관은 인천유일의 국보로 고려시대 대표적인 불교문화재 가운데 하나로 불력(佛力)을 이용해 거란군의 침입을 물리치려했던 조상들의 정성이 가득 담긴 《초조본유가사지론 권 제53(국보 제276호)》을 비롯해 우리나라에 자생하는 약초를 집대성한 향약집성방의 기초가 되는 《향약제집성방  (보물 제1178호)》, 중국에서 전해오는 한의학서를 우리나라 실정에  맞게 편찬한 전문 의학서인 《태산요록(보물 제1179호)》등 귀중한 보물 13점을 소장하고 있다. 이밖에도 의학 전문 박물관으로서 면모를 갖추기 위해 일반고서 5,200여 점과 의료·생활유물 1,000여 점, 근대정부기록외에  국내최다 소장 창간호 등 20,000여 점의  귀중한 자료를 소중하게 간직하고 있다.


no 38 :  인천학생과학관
인천 중구
인천광역시 중구 영종대로277번길 74-10
※ 코로나바이러스감염증-

no 58 :  인하대학교 박물관
인천 미추홀구
인천광역시 미추홀구 인하로 100
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.02.24 ~ 미정)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기인하대학교에서 운영하는 박물관으로 1976년 3월 1일 국내외 역사·문화·민속과 인류학 분야 자료를 모아 전시하고 이를 연구하기 위하여 설립하였다. 1983년까지는 주로 인천과 주변 경기지방의 각종 문헌자료를 모아 연구하였고, 1984년 이후에는 인하대학교의 설립취지 및 지역적 특성을 갖춘 유물과 교통관련 유물을 모으는 데 주력하는 한편, 1994년부터 국내외 민속 및 인류학 자료 수집과 조사 연구에 중점을 두고 있다.


no 59 :  송도컨벤시아
인천 연수구
인천광역시 연수구 센트럴로 123
첨단 유비쿼터스 시설을 갖춘 국제 수준의 컨벤션센터 송도 컨벤시아는 지하1층, 지상4층, 연 건축면적 54,000㎡ 규모로 전시장에는 450개 부스를 설치할 수 있고 회의장은 최대 2,000명을 수용할 수 있다. 또 30∼500명 단위로 최대 23실까지 동시에 운영할 수 있는 미팅룸도 마련됐다. 송도컨벤시아는 시드니 오페라 하우스를 연상케 하는 독특한 외관과 내부 고품격 인테리어 등 국내에서 흔히 볼 수 없는 감각적인 디자인을 자랑하며 각종 방송이나 공연, 패션쇼 등의 촬영지로 각광받고 있다.


no 60 :  심은미술관
인천 강화군
인천광역시 강화군 하점면 강화서로 915-1
심은미술관은 강화군 하점면 이강2리 357번지에 소재한 구 강후초등학교가 2000년 2월 28일 학생감소로 인해 폐교됨으로써, 40년 간 이어온 교육목표와 이념이 단절됨을 안타까워하던 심은 전정우 관장이 강화교육청으로부터 모교를 대부받아 그의 아호를 따서 새롭게 미술관을 설립하게 된것이 계기가 되었다. 구 강후초등학교 제1회 졸업생이자 동문회장이며 그리고 그 동네에서 태어나고 자란 심은 전 정우 관장은 강화에 예술인들을 위한 활동 공간이나 전시공간이 마땅치 않았던 차에 작

no 13 :  광주시청 야외음악당
광주 서구
광주광역시 서구 내방로 111
광주시청 옆 문화광장내에 위치한 야외음악당은 2006년 5월 설립한 이동형 객석 2,000석 규모의 야외음악 공연장이다. 최신 음향장비 및 무대, 조명시설을 갖추고 있으며, 클래식, 국악 등의 음악공연을 비롯한 각종 문화공연 및 전시행사가 열린다.


no 14 :  광주 역사민속박물관
광주 북구
광주광역시 북구 서하로 48-25
사라져 가는 우리 민속자료들을 수집, 전시함으로써 옛조상들의 얼을 깨치고 우리문화의 원형을 후손들에게 전승하고자 1987년 국내 최대 시립박물관으로 개관하였다. 1층은 최신 기법을 적용한 민속전시실, 2층은 조선시대 이후 광주역사를 소개하는 광주 근대역사실이 있다. 광주 근대역사실에는 광주읍성의 모형, 1920년~30년 중창로 거리를 재현해 일제감정기의 생활과 사건, 도시개발과정의 애환과 5·18민주화운동 등의 역사를 소개하고 있다. 주변에 국립광주박물관, 광주시립미술관, 문화예술회관, 광주비엔날레 전시장, 우치공원 등이 있다.


no 15 :  김대중컨벤션센터
광주 서구
광주광역시 서구 상무누리로 30
* 지역 경제 활성화를 추구하는, 김대중컨벤션센터김대중컨벤션센터는 최상급시설의 전시, 컨벤션센터로 천혜의 자연이 살아 숨쉬는 관광 광주에 건립되었다. 2003년 11월 광주전시컨벤션센터로 착공해 2005년 5월 김대중컨벤션센터로 이름을 바꾼 뒤, 같은 해 9월 6일 문을 열었다. 광주를 동북아 무역 중심도시로 육성하여 국가 경쟁력을 강화시키며, 관광산업 등 지역 특화산업의 발전에 견인함으로써 지역결제 활성화 도모에 힘쓰고 있다. 또한, 국제적 상품, 기술, 정보, 문화 교류의 장으로써 지역역량을 강화하고 있다. 그리고 풍부한 노하우를 갖춘 최고의 전문인력과 최고의 서비스와 다양한 편의시설을 갖추었다. * 김대중컨벤션센터의 주요 사업김대중컨벤션센터에서는 국내외 박람회·전시회·회의·세미나 개최, 부동산 관리·운영 및 임대, 기계장비와 용구의 관리·임대, 무역 

no 28 :  고싸움놀이전수회관
광주 남구
광주광역시 남구 고싸움로 2
고싸움놀이는 칠석동(옻돌)마을에서 해마다 음력 정월 10일경부터 2월초하루에 걸쳐 연화돼 오는 놀이로 정월보름과 16일에 절정을 이룬다. 이 마을에서 행해지고 있는 고싸움놀이는 양편이 굵은 줄에 단 "고"를 어깨에 매고 서로 부딪쳐 이편 "고"로 상대편 "고"를 눌러 땅에 닿게하여 승부를 겨루는 놀이다. 예로부터 해마다 음력 정월보름이 되면 재앙을 물리치고 복을 불러들여 마을이 편안하기를 비는 세시풍속으로 당산제를 지내고 "고싸움놀이"를 벌였다. 정월 14일 밤에는 마을 사람들이 마을 뒤쪽에 있는 할아버지 당산고 마을 앞쪽에 있는 할머니 당산에 당산제를 지내고 농악을 치면서 당산고 마을곳곳을 돌며 마당밟이를 하고 나서 16일이 되면 마을 상촌 사람들이 동부로, 하촌사람들은 서부로 편을 갈라서 고싸움놀이를 벌여 승부를 냈다. 고싸움의 "고"라는 말은 "옷고름" "고맺음" "고풀이"등의 예에서 보는 바와 같이 노끈의 한가닥을 늘여 둘그런 모양으로 맺은 것을 지칭한 것이며 "고싸움"이란 2개의 "고"가 서로 맞붙어 싸움을 벌인다는 데서 연유된 것으로 본다. 이 놀이는 1945년을 전후해서 그 자취를 감추어 인면되어 가던 것을 1969년 전남대학교 지춘상교수에 의해 발굴, 재현, 재구성되어 오늘날에 보존, 계승되고 있다. 이 고싸움놀이는 다른 지방 민속놀이보다 강인한 패기와 굳센 힘이 넘쳐 흐르고, 일사불란한 통제력과 강철같은 협동심, 굽힘없는 투지와 끈질긴 인내심을 필수요건으로 하는 가장 남성적인 놀이로서 남도민의 기상과 멋이 깃든 놀이다. 


no 29 :  민들레소극장
광주 동구
광주광역시 동구 동계천로 111
극단  토박이의 전용극장으로 1995년에 개관한 극장이다. 민들레 소극장은 시민 성금에 의한 공간으로, 지역민의 정서에 부응하는 공연만을 하는 곳이다. 광주 민주 항쟁과 지역문제 중심의 연극을 하고 있다.


전체 검색 결과 건수 : 40 건
실제 최종 출력 건수 40
no 1 :

AttributeError: 'NoneType' object has no attribute 'find'

In [9]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설 대전부터'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(5,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 60
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            except IndexError:
                addr = '0'
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) +'//'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
cul = pd.DataFrame()
cul['no']=no2
cul['title']=contents
cul['area_cd']=area_CD2
cul['addr']=addr2
cul['content']=view2

# csv 형태로 저장하기
cul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
cul.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 40 건
실제 최종 출력 건수 40
no 1 :  유성문화원
대전 유성구
대전광역시 유성구 문화원로 46
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.24 ~ 2020.09.20)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기유성문화원은 유성지역의 중추적 문화예술기관으로서 문화예술을 통해 지역주민과 소통하고 화합하며, 지역주민의 평생학습을 위한 문화학교를 운영하고 있다. 유성문화원은 그동안 지역문화 창출의 산실로서 각종 전시회 및 강연회ㆍ지역축제는 물론, 지역의 민속놀이와 문화재 사료의 수집 및 정리, 발간작업 등을 통하여 우리 지역에 산재해 있는 문화유산을 후손들에게 물려주는 일에 최선을 다하고 있다.유성문화원은 역사문화고장 만들기 일환의 향토사 연구 등을 통해 문화유산의 보존과 존통의 현대적 계승 사업을 진행하고 있다. 또한, 전통강좌를 비롯한 48개 강좌운영 및 창의적 프로그램 개발 등 문화예술교육을 통한 문화역량 강화를 꾀하고 있다. 그리고 가족단위 문화예술 향유와 문화봉사활동 프로그램 개발을 통한 여가문화의 질 향상도 추진하고 있다.


no 2 :  이응노 미술관
대전 서구
대전광역시 서구 둔산대로 157
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 2020.09.20)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기대전 이응노미술관은 이 지역이 낳은 세계적인 작가 고암 이응노(1904-1989) 화백의 예술 연구와 전시를 맡아 이 시대 고암 정신을 확장하고 계승할 못적으로 2007년 5월 개관하였다. 이응노의 삶과 예술활동을 재조명하고, 그의 예술세계를 연구함으로써 한국 미술의 발전에 기여하고자 하는 뜻을 내포하고 있다. 대전광역시 서구 만년동 396번지에 있으며, 지하 1층, 지상 2층 건물이다. 박물관 건축설계는 프랑스 건축가 로랑 보두앵이 맡았다. 2007년 5월 3일 개관하였고 개관 기념전으로 ‘고암, 예술의 숲을 거닐다-파리에서 대전으로

no 17 :  대덕문화원
대전 대덕구
대전광역시 대덕구 대전로 1348
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.24 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기대덕문화원은 향토조사 및 발간산업, 문예, 교육, 민속, 축제사업을 운영하여 대전 전통문화의 맥을 이어오고 있다.* 설립일 : 1995년 1월 5일


no 18 :  대덕문예회관(구, 대덕구 평생학습센터)
대전 대덕구
대전광역시 대덕구 대전로 1348
※ 코로나바이러스감염증-19 공지사항※ 내용 : 프로그램 운영중단 (2020.08.24 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기2000년 개관한 대덕구문예회관은 공연장(200석), 전시실(2개)로 구성되어 있으며, 예능학습실, 산학습실,호학습실으로 구성된 강좌실과 도서열람실, 회덕작은도서관으로 되어있는 지역의 유일한 문예회관이다.* 개관일 : 2000년 12월 20일


no 19 :  한밭교육박물관
대전 동구
대전광역시 동구 우암로 96
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기한밭교육박물관은 1992년 7월 10일 교육부의 승인을 받아, 대지 4,198㎡에 연건축면적 2,195㎡으로 9개의 전시실과 3개의 전시장, 야외 전시장 그리고 각종 부대시설을 갖추고 옛날에 사용하던 교과서를 비롯하여 교육관련도서, 교육학습기록, 교원학생서장, 사무용품 등 교육관련유물 총 27,000여점의 자료를 보존, 전시하고 교육과정과 연계하여 학생들의 학습장으로 활용되도록 운영하고 있는 교육박물관이다. 제1전시실에서는 옛날 서당교육의 모습에서부터 구한 말 신식교육이 들어오기까지의 생생한 모습과 많은 자료가 시대, 영역별로 전시하고 있으며, 제2전시실에서는 일제 강점기 시절의 여러가지 아픈 과거인 창씨개명서장, 황국신민서사석 등 황민화 교육의 모습들이 전시되어 있다. 

no 32 :  한남대학교 성지관
대전 대덕구
대전광역시 대덕구 한남로 70
* 지역 시민의 문화 공간, 한남대학교 성지관 *성지관은 교문에서 정면으로 바라다 보이는 건물로 북쪽에 위치하고 있다. 이 건물은 1985년 4월에 착공하여 1986년8월에 준공하였다. 연면적 905.18평에 객석은 1층 1048석, 2층 564석 총 1612석으로 당시 충청권 최대 규모의 강당으로, 채플을 비롯한 각종 학교 행사와 지역 시민의 문화공간으로 활용되기 위해 건축되었다."거룩한 뜻"의 「성지관」으로 명명한 것은 당시 성지관의 주기능이 채플이었기에 채플 시간을 통해 하나님의 거룩한 뜻 이 선포되며, 그분의 거룩한 뜻을 듣는 모든 학생들이 참 신앙을 통한 바른 인격을 형성한 지성인으로서 거룩한 뜻을 키우고 다짐하는 곳이라는 의미이다. 남서쪽 벽면에 "聖志館" 낱자가 가로 세로 각각 2m 크기의 글씨를 볼 수 있는데 이 글씨는 국전초대 작가인 장암 이곤순 씨의 글씨체를 그대로 주물로 제작한 것이다. 성지관 정면에 걸린 십자가는 그 크기가 4m의 대형으로 미술교육과 박병희 교수의 설계로 제작하였다. 교문에서 바라볼 때 정면에 위치함으로써 교문의 십자가상과 더불어 한남의 교육이념이 그리스도 정신에 바탕하고 있음을 가시적으로 나타내는 상징물이라 할 수 있다.


no 33 :  충남대학교 정심화국제문화회관
대전 유성구
대전광역시 유성구 대학로 99
충남대학교에 들어서면 정문을 지나자마자 오른쪽에 독특한 이름의 대형 건물이 있다. ''정심화 국제문화회관’은 정심화홀, 백마홀, 대덕홀이 있다. ‘정심화''는 지난 1990년 김밥을 팔아서 평생 모은 50억원대의 재산을 학교발전을 위하여 충남대학교에 기증한 고 이복순 여사의 법명이다. 故정심화 할머니의 뜻을 담은 이름의 정심화국제회관을 건립하게 되었다. 건설 도중 어려움도 많았지만 주변의 도움으로 2000년에 완공되었다. 정심화홀은 1,817석의 좌석이 있으며 공연, 음악회, 대규모 행사 등에 사용되고 있고 백마홀과 대덕홀은 각각 450,

no 13 :  울산대곡박물관
울산 울주군
울산광역시 울주군 두동면 서하천전로 257
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.25 ~ 2020.09.06)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기대곡박물관은 대곡댐 건설로 불가피하게 수몰된 대곡리 일대에서 출토된 문화유산을 전시, 연구하는 박물관이다.대곡댐 편입부지 발굴조사 당시 하삼정 고분군을 비롯하여 많은 유적이 확인되었으며, 1만 3천여 점의 유물이 출토되어 이를 전시하기 위해 2009년 6월 24일 개관하였다.


no 14 :  약사동 제방유적전시관
울산 중구
울산광역시 중구 종가14길 22-28
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.25 ~ 2020.09.06)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기약사동 제방유적 전시관에 들어서면 거대한 제방의 단면인 사다리꼴의 성토층이 전시장 한쪽 면을 가득 채우고 있다. 약사동 제방은 삼국시대 말에서 통일신라시대 초(6~7세기)에 축조된 수리시설로 다른 고대 수리 유적들과 달리 발굴을 통해 축조시기와 기법이 밝혀진 유적이라는 점에서 그 의미가 남다르다. 2년여에 걸친 발굴조사에서 축조방법을 확인한 결과 제방은 단순히 흙을 쌓아 만든 것이 아니라 현대의 댐처럼 매우 복잡한 구조로 되어있다. 가공된 기초지반 위에 점성이 높은 실트층과 패각류를 깔고, 잎이 달린 가는 나뭇가지를 이용한 부엽공법(敷葉工法) 등 고대토목기법이 사용돼 중요한 학술적 가치를 가진 것으로 평가된다. 태화강 하류에서 채집한 굴 껍질의 탄산칼슘이 수분과 만나 흙을 더욱 단단히 잡을 수 있다는 것을 신라인들이 알고 있었다는 사실이 놀랍고 흥미롭다. 전시관에는 제방 유적 외에도 당시 지역민들의 생활상을 알 수 있는 농기구, 토기 등 출토품들이 전시되어 있으며, 다양한 영상자료를 통해 흥미로운 역사 공부의 기회를 제공한다.


no 15 :  울산울주도서관
울산 울주군
울산광역시 울주군 언양읍 언양로 109

no 4 :  경기과천교육도서관
경기 
경기도 과천시 도서관길 36
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.09.01 ~ 별도안내시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기경기과천교육도서관은 경기도교육청 소속 도서관으로 1984년에 개관하여 지난 30년 동안 수준 높은 자료의 신속한 제공과 다양한 독서활동 및 평생교육 프로그램을 통하여 지식정보센터이자 종합문화 공간의 역할을 다하고 있다. 21세기 "종합정보이용 공간"으로서 지역주민의 지식,정보욕구를 충족시키기 위한 다양한 정보서비스를 제공하고 있으며, 지식정보의 홍수속에서 누구나 쉽게 정보접근이 가능하도록  여러분의  눈높이에 맞는 정보서비스를 제공하고 쾌적한 환경조성과 친절한 봉사정신으로  공공도서관의 역할을 충실히 수행하는데 그 목표를 두고 있다.


no 5 :  성남문화원
경기 성남시
경기도 성남시 수정구 산성대로215번길 7
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.20 ~ 추후공지시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기성남문화원은 시/군/구 단위에 설립된 특별 법인으로 지역문화 발굴 보존과 각종 문화행사를 주최하며, 지방문화원의 육성은 국가 및 지방자치단체의 지원과 육성으로 이루어진다. 


no 6 :  국립여성사전시관
경기 고양시
경기도 고양시 덕양구 화중로104번길 50
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.08 ~ 별도공지시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기국립여성사전시관은 2002년 12월 9일 서울 대방동에서 개관하였다. 12년간의 대방동 시대를 접고 2014년 9월 1일 경기도 고양에서 이전 개관하면서 최첨단 디지털 미디어를 적극적으로 활용하여 반만년 한민족 여성들의 역사와 문화를 한자리에 모았다.상설전시실에서 고대부터 근현대까지 한국의 역사발전에 기여한 여성인물과 업적 소개를 살펴볼 수 있으며 기획전시실은 심도 깊은 여성사 탐구의 기회를

...

no 12 :  창원시립마산박물관
경남 창원시
경상남도 창원시 마산합포구 문신길 105
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.21 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기총 7,668㎡ 전면적이 박물관으로 구성되어 있으며 역사문화교육, 지역사 자료조사 활동, 문화행사 등 다양한 박물관 프로그램을 운영하고 있다. 창원의 고대에서 현대까지의 역사를 한 눈에 볼 수 있는 상설전시실과 주제에 따라 새롭게 구성되는 특별전시실을 관람할 수 있다. 야외전시장에는 최치원관련 13인 시비와 몽고정맷돌, 장승 및 솟대가 전시되어 있어 유익한 볼거리이다.* 개관일 2001년 9월 12일


no 13 :  밀양아리랑아트센터
경남 밀양시
경상남도 밀양시 밀양대공원로 112
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.24 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기밀양아리랑아트센터는 밀양문화재단이 운영하는 다목적 공연장이다. 공연장은 765석의 대공연장과 236석의 소공연장, 연습실(중,대), 출연자 대기실 등으로 구성되어 있고 자체기획 및 대관 공연을 진행하고 있다. 별관인 전시관은 1,261㎡ 규모로 밀양아리랑 상설전시관과 밀양아리랑 전수관, 전시실 등이 있으며, 밀양아리랑상설전시관의 경우는 상시 개방되어 있고, 전시실의 경우는 자체 전시 및 대관 전시를 진행하고 있다.


no 14 :  밀양의열기념관
경남 밀양시
경상남도 밀양시 노상하1길 25-12
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기2018년 3월 7일 약산 김원봉 장군의 생가터에 문을 연 “의열기념관”은 “자신의 목숨을 돌보지 않고 충의(忠義)에 앞장 선 사람”을 뜻하는 “의열(義烈)”의 정신을 후세에 전하고 오늘을 살고 있는 우리 모두에게 독립운동의 참 가치를 보여주기 위해 전

no 32 :  창원문화원
경남 창원시
경상남도 창원시 의창구 용지로239번길 19-22
창원문화원은 창원 지역 고유문화의 계발·연구·조사와 각종 지역 문화 행사의 개최 등을 통하여 지역 주민의 문화생활 질을 높이고, 수준 높은 문화 예술을 보급하여 창원 지역 문화 창달에 이바지하고자 설립되었다. 이러한 설립목표를 기반으로 창원문화원은 지역예술인과 서로 편안하게 조우하여 문화예술진흥, 문화복지 증진을 실감하도록 만들면서 궁극적으로 지역사회발전을 함께 이룰 수 있도록 묵묵히 지원하는 역할을 하고 있다. 창원문화원은 문화체육부 산하 비영리 기관으로 일정한 시설을 갖추고 창원 지역의 문화·사회 교육사업과 향토 문화 연구와 각종 행사 등을 시행하고 있다.창원문화원은 매년 창원 지역의 문화 향토사를 조사 연구하고 있으며, 사료 수집과 보존을 위하여 각종 문화 사업을 시행하고 있다. 또한, 창원 지역에서 각종 문화 행사와 대회 등을 기획 및 공연하고 있다. 매년 문화답사와 향토순례를 기획하여 시행하고 있으며, 문화교실을 운영하여 취미교양강좌로 생활 악기, 하모니카, 플루트, 생활역학, 풍수지리, 속독, 논술, 독서 지도사 과정 등을 개설하여 평생교육에 이바지하고 있다.


no 33 :  국립김해박물관
경남 김해시
경상남도 김해시 가야의길 190
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기* 가야 문화를 만나는 곳, 국립김해박물관 *1998년 7월 29일, 국립김해박물관은 고대국가의 하나인 가야(加耶)의 문화유산을 집대성하기 위해 개관하였다. 가야의 건국설화(建國說話)가 깃든 김해시 구지봉(龜旨峰) 기슭에 자리 잡은 국립김해박물관은 가야의 문화재를 집약 전시하고 있으며, 아울러 부산·경남지역의 선사시대의 문화상과 가야의 성장 기반이 된 변한(弁韓)의 문화유산을 전시하고 있다. 건물 전체를 검은 벽돌로 외장하여 철광석과 숯의 이미지로 철의 왕국 가야를 상징적으로 표

no 52 :  거창박물관
경남 거창군
경상남도 거창군 거창읍 수남로 2181
거창의 역사 유물을 한눈에 살필 수 있는 거창박물관은 한옥구조의 2층 건물로 거창읍 김천리에 자리하고 있다. 박물관의 규모는 2,600평의 부지에 1, 2층 250평의 전시실과, 강당 및 전시실을 갖춘 120여 평의 별관, 그리고 야외전시장으로 이루어져 있다. 박물관을 만들게 된 배경은 이 지역의 독지가인 계림농원 최남식 대표와, 제창의원 김태순 원장 두 사람이 평생 수집한 귀중한 자료들을 당국에 기증하고, 직접 건립운동에 앞장서면서 1988년 5월 거창유물전시관으로 개관되었다.소장되어 있는 유물의 수는 1,200여 점으로 중요소장품은 대동여지도(유형문화재 제275호), 송림사지석조여래좌상(유형문화재 제311호), 정온선생 관복(중요민속자료 제218호), 이보흠선생 실기책판(유형문화재 제248호) 등의 지정문화재와 다수의 중요자료가 소장되어 있다.본관 1층에는 둔마리 벽화고분 모형, 선현유품, 안방, 사랑방, 한약방, 농기구 민예품, 생활용품 등이 전시되어 있고, 본관 2층에는 거창군 연혁ㆍ년표 , 대동여지도, 가야 및 삼국유물, 금속유물, 고려자기, 조선자기 등이 전시되어 있다. 그리고 별관에는 시청각실, 자료실, 특별전시실(2층), 학예연구실이 갖춰져 있다.거창박물관은 둔마리고분벽화 자료를 비롯하여 소장된 자료들의 대부분은 거창 지역성을 강하게 반영하고 있어 지역문화를 이해하는데 중요한 문화공간으로 자리매김하고 있다.


no 53 :  엄홍길전시관
경남 고성군
경상남도 고성군 거류면 거류로 335
이 시대의 젊은이들에게 대자연에서 배운 꿈과 희망, 용기와 도전정신을 일깨워 주기 위해 개관된 엄홍길전시관은 고성출신인 히말라야 영웅 엄홍길의 일생과 1985년부터 16동안 히말라야 8,000m 16좌를 모두 완등하기까지의 과정을 체계적으로 전시하고 있다.


no 54 :  창원시립 마산음악관
경남 창원시
경상남도 창원시 마산합포구 수산2길 35
마산음악관은 마산 출신 음악인과 마산

no 10 :  전주역사박물관
전북 전주시
전라북도 전주시 완산구 쑥고개로 259
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.18. ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기전주역사박물관은 전주시가 설립하여 전주문화사랑회에서 수탁받아 운영하는 박물관으로서 전주의 역사를 통시대적으로 보여주면서 그 문화적 특질과 우수성을 부각시키는 지역사 박물관이다. 특히, 전주지역의 역사문화를 발굴하고 체계화하는 ‘전주학’ 연구의 중심이 되고자 하는 박물관이다.


no 11 :  농업과학관
전북 전주시
전라북도 전주시 완산구 농생명로 300
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.24 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기농업과학관은 지난 1983년부터 설치 운영되고 있는 농업과학기술 교육 및 농촌 진흥사업 종합 홍보관이다. 첨단농업과학 기술과 농경유물 및 새로운 영농자재 전시를 통하여 농업인들에게는 새로운 농업과학기술 정보를 제공하고 젊은 학생과 도시인들에게는 농업의 중요성을 새롭게 인식시키고 있다. 우리 나라를 찾는 외국인 방문자들에게는 우리 나라 농업 현황과 우수성을 한자리에서 살펴볼 수 있도록 하는 농업과학기술 종합전시홍보관으로서의 역할을 수행하고 있다.


no 12 :  전북도립미술관
전북 완주군
전라북도 완주군 구이면 모악산길 111-6
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.20. ~ 2020.09.06.)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기전북도립미술관은 예향 전라북도에 세워진 대표적인 예술문화공간으로 2004년 10월에 개관하였다. 뒤에는 전북의 명산 모악산이, 앞에는 광활한 구이 저수지가 펼쳐져 있어 문화와 자연을 함께 공유 할 수 있는 최적의 위치에 자리 잡아 도민들의 많은 사랑을 받고 있는 곳이다. 다양한 전실의 개최, 미술작품의 수집과 보존, 문화예술교육과 국제교류 등을 통

IndexError: list index out of range

In [12]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설 전북부터'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(14,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 60
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                addr = '0'
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) +'//'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
cul = pd.DataFrame()
cul['no']=no2
cul['title']=contents
cul['area_cd']=area_CD2
cul['addr']=addr2
cul['content']=view2

# csv 형태로 저장하기
cul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
cul.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 104 건
실제 최종 출력 건수 60
no 1 :  국립전북기상과학관
전북 정읍시
전라북도 정읍시 서부산업도로 168-43
※ 코로나바이러스감염증-19 공지사항※ 내용 : 국립전북기상과학관 홈페이지(http://jbsci.kma.go.kr) 공지사항 통해 임시휴관 여부 및 부분개관 확인 후 방문→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기첨단과학산업도시 전라북도 정읍시에 위치한 국립전북기상과학관은 체험교육 중심의 기상과 우주과학이 융합된 특성화 과학관이다. 스토리텔링방식의 기상·우주과학 체험교육 프로그램을 통해 누구나 쉽게 기상과학을 접하고 이해할 수 있다. 특히 자유학년제에 따른 청소년 진로직업 체험, 대학생 대상 직업체험 등 수준별 프로그램을 운영하며 지역교육센터 역할을 수행하고 있다. 주간에는 일기도그리기, 기상캐스터 체험, 기상현상 4D체험, 3차원지구가시화시스템, 기상재해 가상현실(VR)체험 등의 기상과학 프로그램과 밤하늘 시뮬레이션을 통한 계절 별자리 해설, 천체망원경 조작법, 태양관측체험 등의 우주과학 프로그램이 운영된다. 야간에는 계절 별자리, 달, 행성, 딥스카이(성운, 성단) 해설 및 관측 체험교육이 운영된다.


no 2 :  고창고인돌박물관
전북 고창군
전라북도 고창군 고창읍 고인돌공원길 74
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.20 ~ 2020.09.07)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기고창고인돌박물관은 청동기시대의 각종 유물 및 생활상과 세계의 고인돌문화를 한눈에 살펴볼 수 있다. 상설전시관은 전체 전시공간에 대한 관람정보와 상징전시 공간으로 구성하였다. 주제전시실의 전시내용을 압축해서 상징전시로 전시내용에 대한 관심과 흥미를 불러일으킴으로써 적극적인 관람 욕구를 유도하고 관람분위기를 조성하였다. 상징물 옆에는 안내데스크 및 정보검색 시스템을 설치하여 관람객을 위한 사진정보를 제공할 수 있도록 하였다.  전시관3층에 마련된 체험공간에서는 불피우

no 17 :  국립태권도원
전북 무주군
전라북도 무주군 설천면 무설로 1482
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기※ 19_20 한국관광 100선 ※태권도원은 태권도를 향한 신체 단련과 끊임 없는 탐구를 통해 태권도형 인간에 이르는 ''위대한 체(體)·인(認)·지(至)''의 실천 공간으로 태권도 교육과 수련, 체험과 문화교류를 테마로 45가지의 다양한 프로그램을 제공한다. 국내·외 태권도인은 물론 기업, 학생, 일반인 등 태권도를 통한 심신수련을 원하는 사람이라면 누구나 참여 가능하다.


no 18 :  전북도청도서관
전북 전주시
전라북도 전주시 완산구 효자로 225
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.18. ~ 2020.08.31.)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기도민에게 도서 및 정보를 제공하는 공공도서관의 역할외에 2010.11. 문화체육관광부로부터 전라북도대표도서관으로 지정됨에 따라 전북의 종합적인 자료 수집·정리·보존, 도내 공공도서관 지원 및 협력사업 추진, 작은도서관 조성·운영 지원 등 전북의 도서관 발전과 지역주민의 독서 진흥을 위해 다가적인 노력을 기울이고 있다.


no 19 :  진안역사박물관
전북 진안군
전라북도 진안군 진안읍 내사양길 7
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기진안역사박물관은 용담다목적댐 건설로 인해 수몰된 지역의 사라진 삶의 터전을 발굴유물을 통해 보존, 전시하고 지역의 역사와 문화에 관한 자료를 수집, 보관, 전시, 연구하여 대내외적으로 활용되도록 하는 동시에 역사를 통해 소통하고 문화안에서 새로움을 찾는 열린 공간이다. 박물관은 총 2층으로 되어 있다. 1층에는 고고관, 민속관, 세미나실이 있으며, 2층에는 기록관 기획전시관, 학

no 38 :  미당 서정주 시문학관
전북 고창군
전라북도 고창군 부안면 질마재로 2-8
미당시문학관은 미당 서정주 시인의 고향이자 영면지인 전라북도 고창군 부안면 선운리 마을에 세워진 기념관이다.  그의 사후 다음 해인 2001년 가을, 이호종 전 고창군수의 적극적인 도움에 힘입어 개관하였다. 개관일은 11월 3일인데 이는 미당의 중앙고보 재학시절 광주학생의거 지원 시위 사건(1929년, 1930년 2회)을 기념하기 위해서 정해졌다.  이 기념관은 고향의 생가와 묘역 근처에 있어서 더욱 뜻 깊은 공간이며, 폐교된 선운초등학교 봉암분교를 새롭게 단장하여 지었으므로 <친환경>과 <배움>의 건축미학을 지향하고 있다.


no 39 :  조정래 아리랑문학관
전북 김제시
0
대하소설『아리랑』은 우리 문학의 수준을 한 단계 올려 놓았던 작가 조정래의 작품으로, 문학성이나 대중성으로나 사회 각계 각층으로부터 확고한 검증을 받아낸 바 있으며, 작가는 5년이라는 시간을 고스란히 이 작품을 위해 바쳤던 점에서 육필로 써낸 우리의 민족사인 것이다. 이러한 점에서 식민지 역사 속에서 민족의 독립을 위해 피흘렸던 모든 사람들의 숨결을 뜨겁게 느끼고 지역공동체 의식 함양을 위하여 정보기술을 이용, 가상체험을 통한 독립운동 정신을 계승 발전시키고, 민족적 역량을 바로 세우는 계기를 마련하는 교육의 현장으로 활용하고자 한다.* 개관일 : 2003년 5월 16일


no 40 :  고창판소리박물관
전북 고창군
전라북도 고창군 고창읍 동리로 100
고창판소리박물관은 판소리의 이론가이자 개작자, 후원가였던 동리 신재효 및 진채선,  김소희 등 다수의 명창을 기념하고 판소리 전통을 계승 발전시키기 위하여 동리 신재효 선생의 고택 자리에 설립되었다. 고택은 현재 사랑채만 복원되어 남아 있으며  바로 옆에는 동리 국악당이 있다. 판소리 박물관은 이와 같은 판소리의 유형,무형의 자료를 수집.보존.조사.연구.전시.해석함으로써 일반 대중에게 수준높은 판소리 예술의   재교육과 감상의 기회를 제공하고 마

no 6 :  태백산맥문학관
전남 보성군
전라남도 보성군 벌교읍 홍암로 89-19
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.22. ~ 2020.09.04.)(상황에 따라 휴관 기간연장 가능)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기태백산맥문학관은 태백산맥이 관통하는 시대정신인 통일을 염원하는 마음을 담아 북향으로 지어졌으며, 1, 2층 전시실과 5층 전망대를 갖춘 모던 양식으로 작업에는 건축가 김원 씨가 참여했다. 또한 1층 전시실에서 마주 보게 될 높이 8m, 폭 81m에 이르는 ‘원형상-백두대간의 염원’ 벽화는 이종상 교수에 의해 시각화됐으며, 세계 최대, 최초의 야외건식 ‘옹석벽화’로 이미 세간의 관심을 끌고 있다. 문학관에는 1983년 집필을 시작으로 6년 만에 완결하고 이적성 시비로 몸살을 앓았으며, 그 유형무형의 고통을 겪고 분단문학의 최고봉에 올랐던 작가 조정래의 소설『태백산맥』에 대한 자료가 전시되어 있다. ‘소설을 위한 준비와 집필’, ‘소설 『태백산맥』의 탈고’, ‘소설 『태백산맥』 출간 이후’, ‘작가의 삶과 문학 소설 『태백산맥』’이란 장으로 구성되고, 1만 6천여 매 분량의 태백산맥 육필원고를 비롯한 623점의 증여 작품이 전시되어 있다. 부대시설로는 누구나 책을 볼 수 있는 “북 카페”와 작가가 직접 머무르면서 집필활동을 하게 될 “작가의 방”이 있어 타 문학관과 차별을 두고 있다.* 2008년 11월 21일 개관 


no 7 :  여수시민회관
전남 여수시
전라남도 여수시 좌수영로 69
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.22. ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기여수시민회관은 1987년 6월 28일 개관하였으며 960석의 대공연장과  진남문예회관 266석의 소공연장과 전시실 2관을 갖추고 있다.여수의 대표적인 문화시설로  문화예술공연과 각종 강연 및 공공행사를 개최하여 편안한 쉼터로서의 역할을 하고 있다.


no 8

no 24 :  국립해양문화재연구소
전남 목포시
전라남도 목포시 남농로 136
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.22 ~ 2020.09.04)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기* 국내 유일의 해양유물전시관, 국립해양문화재연구소 *목포시 갓바위 문화거리에 자리한 ''국립해양문화재연구소''는 문화재청 소속기관으로, 우리의 소중한 해양 문화유산을 찾고 알리는 국내 하나뿐인 국립해양박물관이다. 2009년 4월 27일 국립해양유물전시관에서 국립해양문화재연구소로 명칭이 변경되었다. 연건평 4,076㎡, 부지 3만 2,357㎡의 부지에 전시관은 지하 1층, 지상 2층 건물로 구성되어 있으며, 총소장품 2,000점에 달한다.전시실은 제1~제4전시실과 야외전시실 및 특별전시실이 있으며, 바다에 잠들었던 옛 배와 다양한 문화재들, 바다 마을 사람들의 삶과 문화, 우리 배-한선(韓船)의 역사 등을 주제로 꾸몄다. 그리고, 야외 해변에는 수상전시관이 조성되어 있는데, 선조들의 해양문화와 발자취가 담긴 문화재를 보면서 그들이 어떻게 바다를 이용하고 개척 하였는가를 느낄 수 있다.* 국립해양문화재연구소의 연사 *1994년 12월 14일 문을 열었으며 1999년 12월 26일 사회교육관과 수장고를 새로 지었다. 전시관을 관리 운영하는 목포해양유물보존처리소는 1981년 8월 25일 국립문화재연구소에 딸린 목포보존처리장으로 시작하여 1990년 1월 3일 국립해양유물전시관으로, 2009년 4월 27일 국립해양문화재연구소로 명칭이 바뀌었다.


no 25 :  목포어린이바다과학관
전남 목포시
전라남도 목포시 삼학로92번길 98
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.22 ~ 2020.09.04)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기2013년 2월 5일 삼학도 근린공원내 목포어린이바다과학관이 개관되었다. 목포어린이바다과학관은 지상 3층 규모로 ‘바다’를 테마로 하여 바다 상상홀, 깊은바다

no 42 :  물사랑 배움터
전남 화순군
전라남도 화순군 사평면 효당로 400
영산강, 섬진강 수계관리위원회에서는 기성세대는 물론 자라나는 세대에게 물의 소중함을 일깨워 영산강과 섬진강을 아름답게 가꾸어가는 지름길을 일러주기 위해 물사랑 배움터를 건립하였다. 물사랑 배움터는 다양한 학습테마로 구성된 물사랑 교육의 새로운 장으로 활용될 것이며 나아가 지역 주민과 더불어 물 보전을 위한 활발한 의견 교류의 장으로서의 역할을 할 예정이다. 물사랑 배움터에서의 체험을 통해 물에 대한 소중함을 새롭게 인식하고 물 사랑과 환경 사랑을 실천할 수 있는 소중한 기회가 되기를 기대한다. 


no 43 :  소전미술관(진도군)
전남 진도군
전라남도 진도군 진도읍 철마길 29
서예 대가인 소전 손재형 선생의 작품과 그를 위해 작업된 국내 거장들의 작품이 전시되어 남도문화의 새로운 자랑거리로 자리매김 하고 있다. 소전미술관에 전시된 작품들은 소전선생의 유족들이 소장하고 있던 부친의 평생역작과 소전 생전을 위해 국내 거장들이 심혈을 기울인 그림들을 진도군에 기증한 것으로, 논산 이은상 시비문 등 소전 손재형 선생의 주옥 같은 작품과 소장 그림 300여점을 교대로 전시하고 있다. 제1전시실에는 선생의 청년시절 작품이, 2전시실에는 문인화, 3전시실에는 장년시설등의 작품, 4전시실에는 양지니, 서희환, 박행보 등 제자들의 작품과 의제 허백련 선생등 거장들의 작품이 전시되어 있다.


no 44 :  보성문화원
전남 보성군
전라남도 보성군 보성읍 송재로 281-11
1967.7월 설립되어 현재까지 7대에 걸쳐 있으며,기구는 사무국과 이사회가 있다. 중점사업은 국가 및 지방자치단체의 시책 홍보 향토문화예술의 개발 진흥, 전통문화의 발굴 보존 및 계승, 향토지 및 지방문화사업자료 편간, 향토사회교육, 국내외 문화단체와의 제휴 협력 등이 있다.


no 45 :  목포문화예술회관
전남 목포시
전라남도 목포시 남농로 102
예향 목포의 대표적 문화공간으로, 1997년 7월 개관하였다. 6개 

no 59 :  손양원목사 순교기념관
전남 여수시
전라남도 여수시 율촌면 산돌길 70-62
솔로몬의 부귀와 지혜보다 욥의 고난과 인내를 더 고귀하게 여겼던 손양원 목사의 깊은 신앙심과 끝없는 사랑을 기리기 위하여 세워진 순교기념관이다. 기념관에는 성화 7점, 사진 액자 98점, 옥중편지 등 8점, 기타유품 5종 100점을 소장하고 있다. 주변에는 손양원목사 기념비 및 3부자 묘지, 모교로 목회했던 성산교회가 있어, 매년 전국에서 기독교 신자들의 순례코스로 널리 알려진 곳이다.  * 손양원목사 생애1902.6.3 경남 함안 출생, 1938년 평양신학교 졸업1939.8.22 여수시 율촌면 애양원교회 부임하여 나환자들을 위해 목회 생활1940.9.25 일본신사참배 거부로 여수경찰서에 투옥되어 6년간 옥고를 치룸 1945.8.17 출옥후 애양원교회에 귀교하여 애양재활병원 원장으로 취임 1948.9 여순사건 당시 장남, 차남인 두아들(동인, 동신)이 안재선 등 폭도들에게 순교당하였으나 폭도인 안재선마저 그리스도 사랑으로 양아들을 삼았으며1950.6.25 사변때 모교에서 끝까지 남아 목회를 하다가 1950.9.13 공산군에 투옥되어 9.28일 여수시 미평동에서 순교당함 


no 60 :  순천 기적의도서관
전남 순천시
전라남도 순천시 해룡면 기적의도서관길 60
순천 기적의 도서관은 다양한 정보 제공을 목적으로 설립되어 어린이들이 원하기만 하면 누구나 쉽게 책을 접할 수 있는 기회를 제공한다. 기적의 도서관은 어린이들이 한 살 아기 때부터 책과 친해지고 도서관과 친해질 기회를 갖도록 운영되며, 책 읽기는 물론, 이야기 들려주기, 노래, 춤, 그림, 영상, 공작, 낭송, 연극 등 많은 활동들이 책 읽기와 연결되어 진행되는 "살아있는 도서관", " 재미가 넘치는 도서관"으로 운영된다.


전체 검색 결과 건수 : 68 건
실제 최종 출력 건수 60
no 1 :  제주교육박물관
제주 제주시
제주특별자치도 제주시 오복4길 25
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임

no 19 :  조안베어뮤지엄
제주 서귀포시
제주특별자치도 서귀포시 대포로 113
제주도 남부 서귀포시의 중문관광단지 내에 위치하고 있는 조안베어뮤지엄은 세계적으로 유명한 테디베어 아티스트인 조안 오의 작업실이자 전시공간이다. 박물관에서는 천연의 소재로 만들어진 테디베어를 만날 수 있으며, 제주 자연의 아름다움과 느긋한 휴식을 취할 수 있는 곳이다. 나지막한 언덕에 있는 조안베어뮤지엄에서는 백여 년 된 오십 그루의 소나무들과 돌담들이 그대로 보존되고 있어 제주 올레를 함께 할 수 있는 곳이다.


no 20 :  피규어뮤지엄 제주
제주 제주시
제주특별자치도 서귀포시 안덕면 한창로 243
키덜트 문화를 한데 모아 전시한''피규어뮤지엄제주''로 초대한다. 남녀노소 누구나 즐겁게 관람하는 아트피규어의 세계 눈을 뗄 수 없는 환상적인 피규어들이 가득한 ''피규어뮤지엄제주'', 액션피규어, 무비 스테츄, 실물크기 영화캐릭터들의 대형 피규어들과 업사이클링 힐링아트 작품들까지 희귀하고 정교한 고퀄리티의 다양한 피규어들이 전시되어 있다. 전 세대가 공감할 수 있도록 영화, 게임, 애니메이션 캐릭터 까지 친근하게 받아들일 수 있도록 많은 피규어들을 접할 수 있다. 특히, 제주도의 지리적 위치와 특성상 여태 접하지 못했던 최고의 캐릭터 산업 키워드인 ''피규어''를 제주도민과 국내외 관광객이 즐길 수 있는 제주도의 대표적인 문화공간이 되도록 노력하겠다.


no 21 :  드라마월드
제주 제주시
0
드라마월드는 1500평의 실내 세트장으로 되어 있는 드라마 스튜디오다. 실제 실내 드라마 세트장과 복원 세트장이 함께 있는 스튜디오로 전시관 내 모든 의상과 소품은 무료로 체험할 수 있다. 단 한복, 웨딩드레스 체험만 유료로 진행하고 있다.(별도 요금 : 5,000원) 드라마월드에서는 서부시대, 유럽거리 등 월드 세트장에서 웨스턴 웨딩, 카우보이, 셀프 웨딩, 모자장수, 왕좌의 게임 체험을 할 수 있는 월드스튜디오와 한국사극 세트장과 한류 세트장, 1900년대 세트장, 테디베어 특별

AttributeError: 'NoneType' object has no attribute 'find'

In [13]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '문화시설 제주'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EB%AC%B8%ED%99%94%EC%8B%9C%EC%84%A4&choiceTagId=651c5b95-a5b3-11e8-8165-020027310001')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1
# cnttt = 50
# page_cnt = math.ceil(cnttt/10)


for j in range(16,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 60
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                addr = '0'
            except:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                else:
                    addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) +'//'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 68 건
실제 최종 출력 건수 60
no 1 :  제주교육박물관
제주 제주시
제주특별자치도 제주시 오복4길 25
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.25 ~ 2020.09.07)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기1995년 4월 29일 개관한 제주교육박물관은 우리 주변에서 사라지거나 잊혀져가는 유형·무형의 교육자료를 발굴하여 영구적으로 보존하고 자라나는 학생들의 참교육을 실현할 수 있는 교육의 장을 열고자 설립되었다. 제주교육에 관한 교과서, 서장, 교구, 문서와 민구 등을 수집하며, 이들을  활용하여 선사시대부터 현대를 걸쳐 각 시대마다 특징적으로 나타낼 수 있도록 자료의 개체 전시, 생동감있는 영상 전시, 그리고 교육 현장을 축소한 디오라마 전시기법을 통해 제주 교육의 과거, 현재, 미래를 제시하고 있다.


no 2 :  제주특별자치도립기당미술관
제주 서귀포시
제주특별자치도 서귀포시 남성중로153번길 15
※ 코로나바이러스감염증-19 공지사항※ 내용 : 제한적 개관 (2020.06.18 ~ 상황 개선 시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기서귀포시 기당미술관은 주요 국내작가들은 물론 국제적인 작가들의 현대미술작품을 소장하고 전시함으로써 세계속의 제주를 선보이면 제주를 찾는 관광객들에게 제주의 빼어난 자연풍광뿐만 아니라 아름다운 미술작품을 감상할 수 있는 공간으로 삼매봉 기슭에 자리하고 있다.


no 3 :  제주 세계자연유산센터
제주 제주시
제주특별자치도 제주시 조천읍 선교로 569-36
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.23 ~ 상황 개선 시까지 / 실내관람 중단)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기제주 세계자연유산센터는 대한민국에서 유일한 유네스코 세계자연유산으로서의 가치를 널리 알리기 위해 조성되었다. 4D영상관에서 제주의 환상적인 자연을 온몸으로 느끼고, 리프트를 타고 제주도 탄생과정 체험

no 21 :  드라마월드
제주 제주시
0
드라마월드는 1500평의 실내 세트장으로 되어 있는 드라마 스튜디오다. 실제 실내 드라마 세트장과 복원 세트장이 함께 있는 스튜디오로 전시관 내 모든 의상과 소품은 무료로 체험할 수 있다. 단 한복, 웨딩드레스 체험만 유료로 진행하고 있다.(별도 요금 : 5,000원) 드라마월드에서는 서부시대, 유럽거리 등 월드 세트장에서 웨스턴 웨딩, 카우보이, 셀프 웨딩, 모자장수, 왕좌의 게임 체험을 할 수 있는 월드스튜디오와 한국사극 세트장과 한류 세트장, 1900년대 세트장, 테디베어 특별전시를 관람할 수 있다.


no 22 :  한국야구명예전당
제주 서귀포시
0
연중 야구 전지훈련장으로 가장 뛰어난 조건을 갖추고 있는 서귀포시는 야구 전지훈련장, 야구인의 마을 등을 시설하여  야구의  메카로 키워 나가겠다는 야심찬 계획을 수립하여 추진하고 있던 중 전 LG 프로야구  이광환감독이 이러한 서귀포시의 의지에 감명을 받아 자신의 운영하고 있는 "야구의 집" 소장품 3,000여 점을 1998년 1월 15일 서귀포시에 기증함으로서 시작되었다.서귀포시에서는 기증자의 뜻을 존중하고 또한 스포츠를 통한 체험 관광을 개발하여 서귀포 관광의 이미지를 제고시키고 지방세수의 확충계기를 마련하는 색다른 경영 수익 사업으로 발전시키는데 그 뜻을 두어 교수 및  전문가의 지원을 받아 대한야구협회 한국야구위원회의 도움을 얻어 3개월여의  준비 작업 끝에  서귀포시 청소년 수련관 1층에 "한국야구 명예 전당"을 조성 개관하게 되었다.


no 23 :  제주항공우주박물관
제주 서귀포시
제주특별자치도 서귀포시 안덕면 녹차분재로 218
이곳에서는 교육과 놀이가 함께 한다. 항공, 공군의 역사는 물론 천문학과 은하계를 돌아보는 아시아 최대 규모의 박물관이다. 39대의 실물 비행기는 아이들에게 인기이고, 박물관 2층에서 바라보는 전망이 좋아 여행자에게도 입소문이 나 있다.


no 24 :  해녀박물관
제주 제주시
제주특별자치도 제주시 구좌읍 해녀박물관길 2

no 40 :  믿거나 말거나 박물관
제주 서귀포시
제주특별자치도 서귀포시 중문관광로110번길 32
리플리의 <믿거나 말거나 박물관(Ripley''s Believe It of Not)>은 탐험가 로버트 리플리(Robert Ripley)를 중심으로 설립된 세계 최대의 박물관 체인으로 제주 서귀포시 중문관광단지 입구에 들어선다. ''믿거나 말거나 박물관''은 갤러리 12개관과 야외공원, 기념품점, 커피숍 등의 시설로 구성되어 있다. 미국, 영국, 캐나다, 멕시코, 태국 등 전세계에 31개가 운영되고 있으며, 제주도 박물관은 11번째 국가이자 32번째이다.이 박물관에는 탐험가 ''로버트 리플리(Robert Ripley)''가 35년간 198개국을 여행하며 찾아낸 수많은 신기하고 기묘한 물건 700여 점이 전시된다. 1989년 독일 통일시 무너진 베를린 장벽, 화성에서 날아온 운석 조각, 유니콘 뿔을 가진 남자 등 그야말로 믿어야 할지 말아야 할지 애매한 이 세상에서 가장 진귀하고 미스터리한 물건들을 전시하여 관람객들에게 놀라우면서도 색다른 재미를 선사한다.모든 전시물들이 많은 스토리와 역사적인 배경을 담고 있어서 어린이들과 청소년들에게 다양한 세계 역사를 체험할 수 있는 기회가 된다는 점이 이 박물관만의 자랑이다.


no 41 :  트랜스포머 오토봇 얼라이언스 (신화월드)
제주 서귀포시
제주특별자치도 서귀포시 안덕면 신화역사로304번길 89
트랜스포머 오토봇 얼라이언스는 단순히 보는 전시가 아닌 가상현실을 접목한 인터랙티브 트레이닝과 미션 수행을 통해 직접 지구의 수호자로 거듭날 수 있는 체험형 전시 이며, 로봇 피규어를 만들던 추억을 지닌 부모세대부터 트랜스포머를 동경하는 아이들까지, 온 가족이 즐길 수 있는 체험 프로그램이 준비되어 있다.


no 42 :  봉황솟대박물관
제주 제주시
제주특별자치도 제주시 한경면 명이2길 66
''봉황''새를 테마로 한 전문 솟대박물관이다. 1,111마리의 ''봉황''새가 앉혀진 ''솟대''들이 실내전시관을 비롯 정원과 박물관